In [1]:
import argparse
import os
import copy
import random
import numpy

import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data
import time

from dataloaders.kitti_loader2 import load_calib, input_options, KittiDepth
from metrics import AverageMeter, Result
import criteria
import helper
import vis_utils

from model import ENet
from model import PENet_C1_train
from model import PENet_C2_train
#from model import PENet_C4_train (Not Implemented)
from model import PENet_C1
from model import PENet_C2
from model import PENet_C4
from basic import *
from thop import profile

In [22]:
class Args:
    def __init__(self):
        self.convolutional_layer_encoding = "xyz"
        self.use_rgb = True
        self.use_d = True
        self.use_g = True
        self.val_h = 352
        self.val_w = 1216
        self.result = os.path.join('..', 'results')
        self.network_model = 'e'
        self.workers = 4
        self.batch_size = 6
        self.round =1
        self.d_lost = False
        self.rgb_lost = False
        self.data_folder = '../dataset/KITTI/kitti_depth/depth'
        self.data_folder_rgb = '../dataset/KITTI/kitti_raw'
        self.jitter = 0.1
        self.not_random_crop = False
        self.random_crop_height = 320
        self.random_crop_width = 1216
        
args = Args()
print(args.convolutional_layer_encoding)

xyz


In [13]:
cuda = torch.cuda.is_available()
if cuda:
    import torch.backends.cudnn as cudnn
    cudnn.benchmark = True
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("=> using '{}' for computation.".format(device))

=> using 'cuda' for computation.


In [16]:
model = ENet(args).to(device)

In [23]:
train_dataset = KittiDepth('train', args)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                                        batch_size=args.batch_size,
                                                        shuffle=True,
                                                        num_workers=args.workers,
                                                        pin_memory=True,
                                                        sampler=None)

success open part_ 1


In [24]:
for i, batch_data in enumerate(train_loader):
    batch_data = {
        key: val.to(device)
        for key, val in batch_data.items() if val is not None
    }             


In [26]:
flops, params = profile(model, inputs=(batch_data,))
print(f"FLOPs: {flops / 1e9} G FLOPS")
print(f"Parameters: {params / 1e6} M paramters")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.ConvTranspose2d'>.
[INFO] Register count_softmax() for <class 'torch.nn.modules.activation.Softmax'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
FLOPs: 4085.795796 G FLOPS
Parameters: 131.762888 M paramters


In [37]:
for idx, module in enumerate(model.named_children()):
    print(f"Module {idx}: {module}")

Module 0: ('geofeature', GeometryFeature())
Module 1: ('rgb_conv_init', Sequential(
  (0): Conv2d(4, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
))
Module 2: ('rgb_encoder_layer1', BasicBlockGeo(
  (conv1): Conv2d(35, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(67, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (downsample): Sequential(
    (0): Conv2d(35, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
))
Module 3: ('rgb_encoder_layer2', BasicBlockGeo(
  (conv1): Conv2d(67, 64, kernel_size=(3

In [13]:
for name, layer in model.named_children():
    if isinstance(layer, nn.Sequential):
        print(layer)

Sequential(
  (0): Conv2d(4, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)
Sequential(
  (0): ConvTranspose2d(1024, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)
Sequential(
  (0): ConvTranspose2d(512, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)
Sequential(
  (0): ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)
Sequential(
  (0): ConvTranspose2d(128, 64, ker